In [ ]:
#DONE 1. demonstrate test locally
    # bonus: set up tensorboard
# 2. demonstrate test at 
# 3. change to smaller ResNet
# 4. demonstrate on a dcm slice
# 5. demonstrate in 3d

In [13]:
import numpy as np
from skimage import transform

In [6]:
a=np.zeros((10,10,10,3))

In [17]:
np.save('img2', img)

In [10]:
img = np.load(r'imgs\img3.npy')

In [15]:
img.shape

(26, 24, 30, 3)

In [14]:
img = transform.downscale_local_mean(img, (10,10,3,1))

In [18]:
a = np.zeros((3,5))

In [2]:
from keras.layers import Input, Add, Dense, Activation, Flatten, Conv3D, MaxPooling3D, ZeroPadding3D, \
    AveragePooling3D, TimeDistributed
from keras import backend as K
from keras_frcnn.RoiPoolingConv import RoiPoolingConv
from keras_frcnn.FixedBatchNormalization import FixedBatchNormalization
from keras.models import Model
from keras.engine import Layer, InputSpec
from keras import initializers, regularizers
from keras import backend as K
from keras.optimizers import Adam
from keras.layers import Input
from keras.callbacks import TensorBoard
from keras_frcnn import config, data_generators
from keras_frcnn import losses as klosses
import keras_frcnn.roi_helpers as roi_helpers
from keras.utils import generic_utils
from functools import partial
import pickle
import random
from keras_frcnn import resnet as nn

Using TensorFlow backend.


In [3]:

class FixedBatchNormalization(Layer):

    def __init__(self, epsilon=1e-3, axis=-1,
                 weights=None, beta_init='zero', gamma_init='one',
                 gamma_regularizer=None, beta_regularizer=None, **kwargs):
        self.supports_masking = True
        self.beta_init = initializers.get(beta_init)
        self.gamma_init = initializers.get(gamma_init)
        self.epsilon = epsilon
        self.axis = axis
        self.gamma_regularizer = regularizers.get(gamma_regularizer)
        self.beta_regularizer = regularizers.get(beta_regularizer)
        self.initial_weights = weights
        super(FixedBatchNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (input_shape[self.axis],)

        self.gamma = self.add_weight(shape,
                                     initializer=self.gamma_init,
                                     regularizer=self.gamma_regularizer,
                                     name='{}_gamma'.format(self.name),
                                     trainable=False)
        self.beta = self.add_weight(shape,
                                    initializer=self.beta_init,
                                    regularizer=self.beta_regularizer,
                                    name='{}_beta'.format(self.name),
                                    trainable=False)
        self.running_mean = self.add_weight(shape, initializer='zero',
                                            name='{}_running_mean'.format(self.name),
                                            trainable=False)
        self.running_std = self.add_weight(shape, initializer='one',
                                           name='{}_running_std'.format(self.name),
                                           trainable=False)

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

        self.built = True

    def call(self, x, mask=None):

        assert self.built, 'Layer must be built before being called'
        input_shape = K.int_shape(x)

        reduction_axes = list(range(len(input_shape)))
        del reduction_axes[self.axis]
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        if sorted(reduction_axes) == range(K.ndim(x))[:-1]:
            x_normed = K.batch_normalization(
                x, self.running_mean, self.running_std,
                self.beta, self.gamma,
                epsilon=self.epsilon)
        else:
            # need broadcasting
            broadcast_running_mean = K.reshape(self.running_mean, broadcast_shape)
            broadcast_running_std = K.reshape(self.running_std, broadcast_shape)
            broadcast_beta = K.reshape(self.beta, broadcast_shape)
            broadcast_gamma = K.reshape(self.gamma, broadcast_shape)
            x_normed = K.batch_normalization(
                x, broadcast_running_mean, broadcast_running_std,
                broadcast_beta, broadcast_gamma,
                epsilon=self.epsilon)

        return x_normed

    def get_config(self):
        config = {'epsilon': self.epsilon,
                  'axis': self.axis,
                  'gamma_regularizer': self.gamma_regularizer.get_config() if self.gamma_regularizer else None,
                  'beta_regularizer': self.beta_regularizer.get_config() if self.beta_regularizer else None}
        base_config = super(FixedBatchNormalization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [4]:
def add_block(type, input_tensor, filters, block_name, strides=(2,2,2), trainable=True, input_shape=None):
    conv_name_base = 'res' + block_name + '_'
    bn_name_base = 'bn' + block_name + '_'

    n_layers = len(filters)
    if n_layers == 3:
        kernels = [1,3,1]
    elif n_layers == 2:
        kernels = [3,3]
    else:
        raise ValueError("Unhandled layer size %d in add_block!" % n_layers)

    bn_layer = partial(FixedBatchNormalization, axis = 4)

    if 'td' in type:
        k_init = 'normal'
    else:
        k_init = 'glorot_uniform'

    for layer in range(n_layers):
        conv_name = conv_name_base + str(layer)
        bn_name = bn_name_base + str(layer)

        conv_layer = partial(Conv3D, filters[layer],
            (kernels[layer], kernels[layer], kernels[layer]),
            kernel_initializer = k_init,
            padding='same', trainable=trainable)

        if type == "identity":
            x = conv_layer(name = conv_name)(x if layer != 0 else input_tensor)
            x = bn_layer(name = bn_name)(x)
        
        elif type == "id-td":
            x = TimeDistributed(conv_layer(),
                name = conv_name)(x if layer != 0 else input_tensor)
            x = TimeDistributed(bn_layer(), name = bn_name)(x)
        
        elif type == "conv":
            x = conv_layer(strides = strides if layer == 0 else (1,1,1),
                name = conv_name)(x if layer != 0 else input_tensor)
            x = bn_layer(name = bn_name)(x)
        
        elif type == "conv-td":
            if layer == 0 and input_shape is not None:
                x = TimeDistributed(conv_layer(
                    strides = strides,
                    input_shape = input_shape),
                    name = conv_name)(input_tensor)
            else:
                x = TimeDistributed(conv_layer(), name = conv_name)(x)
            
            x = TimeDistributed(bn_layer(), name = bn_name)(x)

        else:
            raise ValueError("Unrecognized type %s in add_block" % type)

        if layer != n_layers - 1:
            x = Activation('relu')(x)

    if "id" in type:
        x = Add()([x, input_tensor])
        x = Activation('relu')(x)

    elif type == "conv":
        shortcut = Conv3D(filters[-1], (1,1,1), strides=strides,
                    name=conv_name_base + 'shortcut', trainable=trainable)(input_tensor)
        shortcut = bn_layer(name=bn_name_base + 'shortcut')(shortcut)

        x = Add()([x, shortcut])
        x = Activation('relu')(x)

    elif type == "conv-td":
        shortcut = TimeDistributed(Conv3D(filters[-1], (1,1,1), strides=strides,
                    trainable=trainable, kernel_initializer='normal'), 
                    name=conv_name_base + 'shortcut')(input_tensor)
        shortcut = TimeDistributed(bn_layer(),
                    name=bn_name_base + 'shortcut')(shortcut)

        x = Add()([x, shortcut])
        x = Activation('relu')(x)

    else:
        raise ValueError("Bad type %s in add_block()" % type)

    return x

def add_blocks(x, block_types, nb_filters, block_num, trainable, stride=True):
    for i, block_type in enumerate(block_types):
        x = add_block(block_type, x, nb_filters,
            block_name=str(block_num)+chr(ord('a')+i),
            trainable=trainable, strides=(2,2,2) if stride else (1,1,1))
    return x

In [14]:
K.set_image_dim_ordering('th')
img_input = Input(shape=(3, 256, 240, 5))
x = ZeroPadding3D((3,3,3))(img_input)
x = Conv3D(64, (7,7,7), strides=(2,2,2), name='conv1', trainable=True)(x)
x = FixedBatchNormalization(axis=4, name='bn1')(x)
x = Activation('relu')(x)
x = MaxPooling3D((3,3,3), strides=(2,2,2))(x)
x = add_blocks(x, ('conv' + ' identity').split(),
    nb_filters=[64, 64, 256], block_num=2, trainable=True, stride=False)
x = add_blocks(x, ('conv' + ' identity').split(),
    nb_filters=[128, 128, 512], block_num=3, trainable=True)
x = add_blocks(x, ('conv' + ' identity').split(),
    nb_filters=[256, 256, 1024], block_num=4, trainable=True)
x = Conv3D(512, (3,3,3), padding='same', activation='relu', kernel_initializer='normal', name='rpn_conv1')(x)

x_class = Conv3D(6, (1,1,1), activation='sigmoid', kernel_initializer='uniform', name='rpn_out_class')(x)
x_regr = Conv3D(33, (1,1,1), activation='linear', kernel_initializer='zero', name='rpn_out_regress')(x)
model_rpn = Model(img_input, [x_class, x_regr])
model_rpn.compile(optimizer=Adam(lr=1e-5), loss=[klosses.rpn_loss_cls(6), klosses.rpn_loss_regr(10)])
model_rpn.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 3, 256, 240, 5 0                                            
____________________________________________________________________________________________________
zero_padding3d_2 (ZeroPadding3D) (None, 3, 262, 246, 1 0           input_2[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv3D)                   (None, 64, 128, 120,  65920       zero_padding3d_2[0][0]           
____________________________________________________________________________________________________
bn1 (FixedBatchNormalization)    (None, 64, 128, 120,  12          conv1[0][0]                      
___________________________________________________________________________________________

In [6]:
all_imgs, classes_count, class_mapping = data_generators.get_data("train_list.txt")

Parsing annotation files
Training images per class:
{'bg': 0, 'cyst': 2, 'hcc': 2}
Num classes (including bg) = 3


In [7]:
def get_config_obj(class_mapping, config_filename):
    C = config.Config()
    C.rot_90 = False
    C.model_path = "model_frcnn.hdf5"
    C.num_rois = 4
    C.network = "resnet50"
    C.base_net_weights = None
    C.class_mapping = class_mapping

    with open(config_filename, 'wb') as config_f:
        pickle.dump(C,config_f)
        print('Config has been written to %s, and can be loaded when testing' % config_filename)

    return C

def get_data_gens(all_imgs, classes_count, C):
    """Get train/val generators"""
    random.shuffle(all_imgs)

    train_imgs = [s for s in all_imgs if s['imageset'] == 'trainval']
    val_imgs = [s for s in all_imgs if s['imageset'] == 'test']

    print('Num train samples {}'.format(len(train_imgs)))
    print('Num val samples {}'.format(len(val_imgs)))

    data_gen_train = data_generators.get_anchor_gt(train_imgs, classes_count, C, nn.get_img_output_length, mode='train')
    data_gen_val = data_generators.get_anchor_gt(val_imgs, classes_count, C, nn.get_img_output_length, mode='val')

    return data_gen_train, data_gen_val


In [8]:
C = get_config_obj(class_mapping, "config.pickle")

Config has been written to config.pickle, and can be loaded when testing


In [9]:
data_gen_train, data_gen_val = get_data_gens(all_imgs, classes_count, C)

Num train samples 2
Num val samples 1


In [10]:
X, Y, img_data = next(data_gen_train)

Checkpoint: GT box coordinates
./imgs/img1.npy: anchor size 1 out of 3
./imgs/img1.npy: anchor size 2 out of 3
./imgs/img1.npy: anchor size 3 out of 3


In [16]:
X=X.transpose([0,4,1,2,3])
X.shape

In [12]:
Y[0]=Y[0].transpose([0,4,1,2,3])
Y[0].shape

(1, 16, 15, 1, 6)

In [18]:
Y[1]=Y[1].transpose([0,4,1,2,3])
Y[1].shape

(1, 33, 16, 15, 1)

In [ ]:
loss_rpn = model_rpn.train_on_batch(X, Y)

In [8]:
import tensorflow as tf
from tensorflow.python import debug as tf_debug
sess = tf.Session()
sess = tf_debug.LocalCLIDebugWrapperSession(sess)
sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)

In [3]:
import os
import keras
import train_frcnn
import importlib

Using TensorFlow backend.


In [21]:
importlib.reload(train_frcnn)

<module 'train_frcnn' from '/Users/clintonwang/Documents/Work/voi-detection/train_frcnn.py'>

In [18]:
x_dim = 50
y_dim = 50
z_dim = 3
dim_tuple = (z_dim, x_dim, y_dim)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
# Generate dummy data
train_size = 10
import numpy as np
data = np.random.random((train_size, z_dim, x_dim, y_dim))
labels = np.random.randint(2, size=(train_size, 1))

# Convert labels to categorical one-hot encoding
one_hot_labels = keras.utils.to_categorical(labels, num_classes=2)

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, one_hot_labels, epochs=10, batch_size=32)

Epoch 1/10
10/10 [==============================] - 11s - loss: 1.7511 - acc: 0.2000
Epoch 2/10
10/10 [==============================] - 1s - loss: 0.7666 - acc: 1.0000
Epoch 3/10
10/10 [==============================] - 1s - loss: 0.7474 - acc: 1.0000
Epoch 4/10
10/10 [==============================] - 1s - loss: 0.7412 - acc: 1.0000
Epoch 5/10
10/10 [==============================] - 1s - loss: 0.7357 - acc: 1.0000
Epoch 6/10
10/10 [==============================] - 1s - loss: 0.7305 - acc: 1.0000
Epoch 7/10
10/10 [==============================] - 1s - loss: 0.7254 - acc: 1.0000
Epoch 8/10
10/10 [==============================] - 1s - loss: 0.7205 - acc: 1.0000
Epoch 9/10
10/10 [==============================] - 1s - loss: 0.7156 - acc: 1.0000
Epoch 10/10
10/10 [==============================] - 1s - loss: 0.7108 - acc: 1.0000
